In [ ]:
import random
import json
import numpy as np
import math
import time # Para medir tiempos si es necesario

from deap import base, creator, tools, algorithms
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle # Para dibujar los radios de cobertura

# --- 1. Cargar y Preparar Datos ---
print(f"[{time.strftime('%H:%M:%S')}] Iniciando script...")
print(f"[{time.strftime('%H:%M:%S')}] Cargando datos desde 'instancia_problema.json'...")
try:
    with open(r"C:\Users\dalon\SynologyDrive\Uni\2024-2025\HaCoBu\HaCoBu\navegacion_maritima\instancia_problema.json") as f:
        datos_problema = json.load(f)
    print(f"[{time.strftime('%H:%M:%S')}] Datos cargados correctamente.")
except FileNotFoundError:
    print("Error: El archivo 'instancia_problema.json' no se encontró.")
    exit()
except json.JSONDecodeError:
    print("Error: El archivo 'instancia_problema.json' no tiene un formato JSON válido.")
    exit()

N_VIVIENDAS = datos_problema.get('N')
M_RADIO_SUPERMERCADO = datos_problema.get('M')
coordenadas_viviendas_raw = datos_problema.get('viviendas')

if N_VIVIENDAS is None or M_RADIO_SUPERMERCADO is None or coordenadas_viviendas_raw is None:
    print("Error: Faltan claves 'N', 'M' o 'viviendas' en el JSON.")
    exit()

coordenadas_viviendas = [tuple(map(int, coord)) for coord in coordenadas_viviendas_raw]

if not coordenadas_viviendas:
    print("Error: La lista de 'viviendas' está vacía.")
    exit()
if len(coordenadas_viviendas) != N_VIVIENDAS:
    print(f"Advertencia: N={N_VIVIENDAS} pero se encontraron {len(coordenadas_viviendas)} pares de coordenadas. Ajustando N.")
    N_VIVIENDAS = len(coordenadas_viviendas)

print(f"[{time.strftime('%H:%M:%S')}] Procesando datos de entrada...")
max_x_vivienda = 0
max_y_vivienda = 0
if N_VIVIENDAS > 0:
    for x, y in coordenadas_viviendas:
        if x > max_x_vivienda: max_x_vivienda = x
        if y > max_y_vivienda: max_y_vivienda = y
else: # No debería ocurrir si el JSON es válido para el problema
    print("Advertencia: No hay viviendas definidas. Estableciendo tamaño de cuadrícula por defecto.")

GRID_WIDTH = max_x_vivienda + 1
GRID_HEIGHT = max_y_vivienda + 1
NUM_CELDAS_TOTAL = GRID_WIDTH * GRID_HEIGHT

print(f"[{time.strftime('%H:%M:%S')}] Problema 1: Cobertura Mínima de Hogares con Supermercados")
print(f"    Número de viviendas (N): {N_VIVIENDAS}")
print(f"    Radio de acción Manhattan (M_RADIO): {M_RADIO_SUPERMERCADO}")
print(f"    Dimensiones de la cuadrícula inferidas: {GRID_WIDTH} (ancho) x {GRID_HEIGHT} (alto)")
print(f"    Número total de celdas (tamaño del cromosoma): {NUM_CELDAS_TOTAL}")

COSTO_POR_SUPERMERCADO = 1.0
PENALIZACION_VIVIENDA_NO_CUBIERTA = 1000 * COSTO_POR_SUPERMERCADO * (N_VIVIENDAS + 1) # Asegurar que sea muy alta


# --- 2. Definición del Algoritmo Genético con DEAP ---
print(f"[{time.strftime('%H:%M:%S')}] Configurando Algoritmo Genético con DEAP...")
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("attr_bool", random.randint, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, NUM_CELDAS_TOTAL)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

def distancia_manhattan(p1, p2):
    return abs(p1[0] - p2[0]) + abs(p1[1] - p2[1])

eval_count = 0 # Contador global para las evaluaciones de fitness

def evaluar_cobertura_supermercados(individual):
    global eval_count
    eval_count += 1
    if eval_count % 50 == 0: # Imprimir cada 50 evaluaciones
         print(f"[{time.strftime('%H:%M:%S')}] Evaluando individuo #{eval_count}...")

    ubicaciones_supermercados_propuestas = []
    costo_construccion_actual = 0
    for i in range(GRID_HEIGHT):
        for j in range(GRID_WIDTH):
            idx = i * GRID_WIDTH + j
            if individual[idx] == 1:
                ubicaciones_supermercados_propuestas.append((j, i))
                costo_construccion_actual += COSTO_POR_SUPERMERCADO

    if not ubicaciones_supermercados_propuestas:
        fitness_val = (N_VIVIENDAS * PENALIZACION_VIVIENDA_NO_CUBIERTA) + costo_construccion_actual
        return (fitness_val,)

    viviendas_no_cubiertas = 0
    for coord_vivienda in coordenadas_viviendas:
        cubierta = False
        for coord_supermercado in ubicaciones_supermercados_propuestas:
            if distancia_manhattan(coord_vivienda, coord_supermercado) <= M_RADIO_SUPERMERCADO:
                cubierta = True
                break
        if not cubierta:
            viviendas_no_cubiertas += 1

    fitness_val = (viviendas_no_cubiertas * PENALIZACION_VIVIENDA_NO_CUBIERTA) + costo_construccion_actual
    return (fitness_val,)

toolbox.register("evaluate", evaluar_cobertura_supermercados)
toolbox.register("mate", tools.cxTwoPoint)
# indpb es la probabilidad de que cada gen mute.
# Un valor típico es 1/longitud_del_cromosoma. Si el cromosoma es muy largo, indpb debe ser pequeño.
# Si NUM_CELDAS_TOTAL es 40000, 1/40000 es 0.000025.
# Un indpb de 0.01 o 0.02 podría ser muy alto si cada bit tiene esa probabilidad.
# tools.mutFlipBit muta cada atributo con una probabilidad independiente indpb.
# Aumentar la probabilidad de mutación general (mutpb) y mantener indpb bajo es una estrategia.
# O usar una mutación que afecte a un número fijo de bits.
# Para este ejemplo, usaré un indpb bajo.
toolbox.register("mutate", tools.mutFlipBit, indpb=max(0.001, 2.0/NUM_CELDAS_TOTAL if NUM_CELDAS_TOTAL > 0 else 0.01) )
toolbox.register("select", tools.selTournament, tournsize=3)
print(f"[{time.strftime('%H:%M:%S')}] Configuración de DEAP completada.")

# --- 3. Configuración y Ejecución del Algoritmo ---
def ejecutar_ag_cobertura():
    global eval_count # Resetear contador para cada ejecución si se llama múltiples veces
    eval_count = 0

    # Para pruebas rápidas, reduce pop_size y ngen.
    # Para una búsqueda real, necesitarás valores mayores.
    pop_size = 50   # Reducido para prueba
    cxpb = 0.8
    mutpb = 0.2     # Probabilidad de que un individuo sea mutado
    ngen = 50       # Reducido para prueba (ej. 50-200 para prueba, 500+ para búsqueda seria)

    print(f"[{time.strftime('%H:%M:%S')}] Parámetros del AG: Pop_size={pop_size}, CXPB={cxpb}, MUTPB={mutpb}, NGEN={ngen}")
    
    print(f"[{time.strftime('%H:%M:%S')}] Creando población inicial de {pop_size} individuos...")
    start_time_pop = time.time()
    pop = toolbox.population(n=pop_size)
    end_time_pop = time.time()
    print(f"[{time.strftime('%H:%M:%S')}] Población creada en {end_time_pop - start_time_pop:.2f} segundos.")
    
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values[0])
    stats.register("avg", np.mean)
    stats.register("std", np.std)
    stats.register("min", np.min)
    stats.register("max", np.max)

    print(f"[{time.strftime('%H:%M:%S')}] Iniciando evolución (algorithms.eaSimple)...")
    print(f"    Esto implicará la evaluación inicial de {pop_size} individuos, y luego {ngen} generaciones.")
    print(f"    Se imprimirá un mensaje cada {eval_count_print_interval if 'eval_count_print_interval' in locals() else 50} evaluaciones de fitness y estadísticas por generación si verbose=True.")

    start_time_ea = time.time()
    pop, logbook = algorithms.eaSimple(pop, toolbox, cxpb, mutpb, ngen,
                                       stats=stats, halloffame=hof, verbose=True)
    end_time_ea = time.time()
    print(f"[{time.strftime('%H:%M:%S')}] Evolución completada en {end_time_ea - start_time_ea:.2f} segundos.")

    mejor_individuo_binario = hof[0]
    mejor_fitness_valor = mejor_individuo_binario.fitness.values[0]

    ubicaciones_supermercados_optimas = []
    costo_construccion_final = 0
    for i in range(GRID_HEIGHT):
        for j in range(GRID_WIDTH):
            idx = i * GRID_WIDTH + j
            if mejor_individuo_binario[idx] == 1:
                ubicaciones_supermercados_optimas.append({"x": j, "y": i})
                costo_construccion_final += COSTO_POR_SUPERMERCADO
    
    viviendas_no_cubiertas_final = 0
    if ubicaciones_supermercados_optimas:
        for coord_vivienda in coordenadas_viviendas:
            cubierta = False
            for coord_supermercado_dict in ubicaciones_supermercados_optimas:
                coord_supermercado = (coord_supermercado_dict["x"], coord_supermercado_dict["y"])
                if distancia_manhattan(coord_vivienda, coord_supermercado) <= M_RADIO_SUPERMERCADO:
                    cubierta = True
                    break
            if not cubierta:
                viviendas_no_cubiertas_final +=1
    else:
        viviendas_no_cubiertas_final = N_VIVIENDAS

    print("\n--- Resultados Problema 1: Cobertura Mínima ---")
    print(f"Valor de Fitness del Mejor Individuo: {mejor_fitness_valor}")
    print(f"Costo de Construcción (Número de Supermercados): {costo_construccion_final}")
    print(f"Número de Viviendas No Cubiertas: {viviendas_no_cubiertas_final}")
    print(f"Número de Supermercados Construidos: {len(ubicaciones_supermercados_optimas)}")
    
    print(f"[{time.strftime('%H:%M:%S')}] Guardando solución en JSON...")
    solucion_json = {
        "problema": "Cobertura Minima de Hogares con Supermercados de Radio Limitado",
        "parametros_entrada": {
            "N_viviendas": datos_problema.get('N'),
            "M_radio_supermercado": datos_problema.get('M'),
            "dimensiones_cuadricula_inferidas": f"{GRID_WIDTH}x{GRID_HEIGHT}",
            "costo_por_supermercado_asumido": COSTO_POR_SUPERMERCADO
        },
        "costo_total_fitness": mejor_fitness_valor, # Esto incluye penalizaciones
        "costo_real_construccion": costo_construccion_final,
        "viviendas_no_cubiertas_final": viviendas_no_cubiertas_final,
        "numero_supermercados_optimo": len(ubicaciones_supermercados_optimas),
        "ubicaciones_supermercados_optimas": ubicaciones_supermercados_optimas
    }
    try:
        with open('solucion_problema1.json', 'w') as f_out:
            json.dump(solucion_json, f_out, indent=2)
        print(f"[{time.strftime('%H:%M:%S')}] Solución guardada en 'solucion_problema1.json'")
    except IOError:
        print("Error: No se pudo escribir el archivo JSON de solución.")

    # --- 5. Graficar la Solución ---
    if ubicaciones_supermercados_optimas: # Solo graficar si hay supermercados
        print(f"[{time.strftime('%H:%M:%S')}] Preparando gráfico de la solución...")
        try:
            fig, ax = plt.subplots(figsize=(15, 12)) # Ventana grande

            v_x = [coord[0] for coord in coordenadas_viviendas]
            v_y = [coord[1] for coord in coordenadas_viviendas]
            ax.scatter(v_x, v_y, c='blue', label=f'Viviendas ({N_VIVIENDAS})', marker='o', s=30, zorder=2, alpha=0.7)

            super_x_opt = [s["x"] for s in ubicaciones_supermercados_optimas]
            super_y_opt = [s["y"] for s in ubicaciones_supermercados_optimas]
            ax.scatter(super_x_opt, super_y_opt, c='red', label=f'Supermercados ({len(ubicaciones_supermercados_optimas)})', marker='s', s=100, zorder=3, edgecolors='black')

            for s_loc_dict in ubicaciones_supermercados_optimas:
                s_x, s_y = s_loc_dict["x"], s_loc_dict["y"]
                diamond_visual_verts = [
                    (s_x, s_y + M_RADIO_SUPERMERCADO + 0.5),
                    (s_x + M_RADIO_SUPERMERCADO + 0.5, s_y),
                    (s_x, s_y - M_RADIO_SUPERMERCADO - 0.5),
                    (s_x - M_RADIO_SUPERMERCADO - 0.5, s_y),
                    (s_x, s_y + M_RADIO_SUPERMERCADO + 0.5) 
                ]
                polygon = plt.Polygon(diamond_visual_verts, closed=True, edgecolor='darkred', facecolor='red', alpha=0.1, zorder=1)
                ax.add_patch(polygon)

            ax.set_xlabel("Coordenada X de la Cuadrícula", fontsize=12)
            ax.set_ylabel("Coordenada Y de la Cuadrícula", fontsize=12)
            ax.set_title(f"Solución AG: {len(ubicaciones_supermercados_optimas)} Supermercados para Cubrir {N_VIVIENDAS - viviendas_no_cubiertas_final}/{N_VIVIENDAS} Viviendas\nCosto Construcción: {costo_construccion_final} (Radio Manhattan: {M_RADIO_SUPERMERCADO})", fontsize=14)
            ax.legend(loc="upper right")
            ax.grid(True, linestyle=':', alpha=0.5)
            ax.set_aspect('equal', adjustable='box')
            
            padding_x = GRID_WIDTH * 0.05 if GRID_WIDTH > 0 else 5
            padding_y = GRID_HEIGHT * 0.05 if GRID_HEIGHT > 0 else 5
            ax.set_xlim(min(v_x)-padding_x if v_x else -0.5, max(v_x)+padding_x if v_x else GRID_WIDTH -0.5)
            ax.set_ylim(min(v_y)-padding_y if v_y else -0.5, max(v_y)+padding_y if v_y else GRID_HEIGHT -0.5)
            # ax.invert_yaxis() # Descomentar si (0,0) debe estar arriba a la izquierda

            plt.tight_layout()
            print(f"[{time.strftime('%H:%M:%S')}] Mostrando gráfico...")
            plt.show()
        except ImportError:
            print(f"[{time.strftime('%H:%M:%S')}] Matplotlib no está instalado. No se pudo generar el gráfico.")
        except Exception as e:
            print(f"[{time.strftime('%H:%M:%S')}] Ocurrió un error al graficar: {e}")
    else:
        print(f"[{time.strftime('%H:%M:%S')}] No se encontraron ubicaciones de supermercados para graficar.")


    return pop, logbook, hof

if __name__ == "__main__":
    print(f"=====================================================================")
    print(f"  Iniciando Ejecución del Algoritmo Genético para Problema 1       ")
    print(f"=====================================================================")
    start_total_time = time.time()
    ejecutar_ag_cobertura()
    end_total_time = time.time()
    print(f"=====================================================================")
    print(f"  Ejecución Total Finalizada en {end_total_time - start_total_time:.2f} segundos.                    ")
    print(f"=====================================================================")

[23:18:52] Iniciando script...
[23:18:52] Cargando datos desde 'instancia_problema.json'...
[23:18:52] Datos cargados correctamente.
Advertencia: N=200 pero se encontraron 2000 pares de coordenadas. Ajustando N.
[23:18:52] Procesando datos de entrada...
[23:18:52] Problema 1: Cobertura Mínima de Hogares con Supermercados
    Número de viviendas (N): 2000
    Radio de acción Manhattan (M_RADIO): 10
    Dimensiones de la cuadrícula inferidas: 200 (ancho) x 200 (alto)
    Número total de celdas (tamaño del cromosoma): 40000
[23:18:52] Configurando Algoritmo Genético con DEAP...
[23:18:52] Configuración de DEAP completada.
  Iniciando Ejecución del Algoritmo Genético para Problema 1       
[23:18:52] Parámetros del AG: Pop_size=50, CXPB=0.8, MUTPB=0.2, NGEN=50
[23:18:52] Creando población inicial de 50 individuos...
[23:18:54] Población creada en 1.87 segundos.
[23:18:54] Iniciando evolución (algorithms.eaSimple)...
    Esto implicará la evaluación inicial de 50 individuos, y luego 50 gene